<a href="https://colab.research.google.com/github/tanaken7/contextual-bandit/blob/master/notebooks/gamiew_training_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
from google.cloud import bigquery
client = bigquery.Client('spartan-production')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd "/content/drive/My Drive/Colab Notebooks/Gamiew送客モデル"
%ls

/content/drive/My Drive/Colab Notebooks/Gamiew送客モデル
 200628_1000000_train_df.csv
 bq-results-20200628-152905-ujmer2ctzyd0.csv
 bq-results-20200629-133445-lhp1usoefnfn.csv
 bq-results-20200629-134202-674soryz6huh.csv
 bq-results-20200629-142639-cjrddie7y8w5.csv
'[Gamiew送客モデル] トレーニングのテスト.ipynb'


In [5]:
# ライブラリのインポート
import pandas as pd
import datetime

In [6]:
# 基準日を設定
base_date = '2020-06-08'

# レンジを設定
range_days = 1

# 対象ユーザー数を設定
user_count = 1000000

# ジャンルを設定
genres = {
    'action':'アクション',
    'tps_fps':'TPS・FPS',
    'novel':'ノベル',
    'card':'カード',
    'casual':'カジュアル',
    'simulation':'シミュレーション',
    'strategy':'ストラテジー',
    'sports':'スポーツ',
    'puzzle':'パズル',
    'board_table':'ボード・テーブル',
    'music':'音楽',
    'racing':'レーシング',
    'action_rpg':'アクションRPG',
    'command_rpg':'コマンドRPG',
    'mmorpg':'MMORPG',
    'shooting':'シューティング',
    'gps':'位置情報ゲーム',
    'quiz':'クイズ',
    'romance':'恋愛',
    'fight':'格闘',
}

# Gamiewのゲームを設定
gamiew_games = {
    'dqw':'11',
    'stella_arcana':'51',
    'epick_seven':'58',
    'kings_raid':'99',
    'mitrasphere':'551'
}

In [7]:
# BigQueryでqueryを実行しdataframeで出力
def exec_query(query):
    return pd.read_gbq(query, project_id="spartan-production", dialect="standard")

# 基準日に訪問したユーザー一人ひとりのレコードに対して、
# 基準日からレンジの期間のジャンル別Game8PVのカラムと、
# 基準日からレンジの期間のジャンル別GamiewCVのカラム
# を持ったdataframeを定義
def generate_train_df(base_date, range_days, genres, exec_query):
    # ジャンル部分のカラムを定義
    cols1 = '\n'.join(["when '"+v+"' then '"+k+"'" for k, v in genres.items()])
    cols2 = '\n'.join(['sum('+k+') as '+k+',' for k, v in genres.items()])[:-1]
    cols3 = '\n'.join(["case c.name when '"+v+"' then 1 else 0 end as "+k+"," for k, v in genres.items()])[:-1]
    cols4 = '\n'.join(["when '"+k+"' then "+v for k, v in gamiew_games.items()])
    cols5 = '\n'.join(["'"+k+"'," for k, v in gamiew_games.items()])[:-1]
    cols6 = '\n'.join(["null as "+k+"," for k, v in genres.items()])
    cols7 = '\n'.join(["sum("+k+") over (partition by unique_id order by time) as "+k+"," for k, v in genres.items()])
    cols8 = ','.join(gamiew_games.values())

    query = """
        with gamiew_genre as (
            select
                *
            from
                external_query(
                    "gabeeee-production.asia-northeast1.connect-cloud-sql",
                    '''
                    select
                        concat(tr.game_id) as gamiew_game_id,
                        case t.name
                            {cols1}
                        end as genre
                    from
                        tag_relationships as tr
                    inner join games as g on g.id = tr.game_id
                    inner join tags as t on t.id = tr.tag_id
                    where t.type = 'genre'
                    and g.id in ({cols8})
                    '''
                )
        ), game8_genre as (
            select
                game_id,
                {cols2}
            from
                external_query(
                    "spartan-production.asia-northeast1.game8",
                    '''
                    select
                        concat(g.id) as game_id,
                        {cols3}
                    from
                        categories as c
                    inner join categories_games as cg on cg.category_game_id = c.id
                    inner join games as g on g.id = cg.game_id
                    where g.published = 1
                    '''
                )
            group by game_id
        ), base_user as (
            select
                distinct unique_id
            from
                `spartan-production.game8_s3.access_logs`
            where _PARTITIONDATE = date_add(date "{base_date}", interval 1 day)
            limit {user_count}
        ), base_access_logs as (
            select
                *
            from
                `spartan-production.game8_s3.access_logs`
            inner join game8_genre as g8g on g8g.game_id = json_extract_scalar(query_string, '$.game_id')
            where _PARTITIONDATE between date_add(date "{base_date}", interval 1 day) and date_add(date "{base_date}", interval {range_days} day)
            and unique_id in (select * from base_user)
        ), base_click_logs as (
            select
                concat(case data_value
                    {cols4}
                end) as gamiew_game_id,
                json_extract_scalar(query_string, '$.archive_id') as archive_id,
                json_extract_scalar(query_string, '$.game_id') as game_id,
                *
            from
                `spartan-production.game8_s3.click_logs`
            where _PARTITIONDATE between date_add(date "{base_date}", interval 1 day) and date_add(date "{base_date}", interval {range_days} day)
            and unique_id in (select * from base_user)
            and data_value in (
                {cols5}
            )
        ), click_data as (
            select
                distinct
                    unique_id,
                    archive_id,
                    time,
                    format_datetime("%H", time) as hour,
                    null as visit_days,
                    {cols6}
                    gamiew_genre.*,
                    1 as cost
            from
                base_click_logs
            full join gamiew_genre using (gamiew_game_id)
            where archive_id is not null
        ), no_click_data_without_genre as (
            select
                unique_id,
                archive_id,
                time,
                format_datetime("%H", time) as hour,
                dense_rank() over (partition by unique_id order by date(time)) as visit_days,
                {cols7}
                concat(null) as gamiew_game_id,
            from (
                select
                    unique_id,
                    json_extract_scalar(query_string, '$.archive_id') as archive_id,
                    max(time) as time,
                    {cols2}
                from
                    base_access_logs
                group by unique_id, archive_id
            )
        ), no_click_data as (
            select
                *
            from
                no_click_data_without_genre
            cross join (
                select
                    distinct genre,
                    0 as cost
                from
                    gamiew_genre
            )
        ), teacher_data_with_blank as (
            (
                select
                    *
                from
                    click_data
            ) union all (
                select
                    *
                from
                    no_click_data
            )
        )
        select
            *
        from
            teacher_data_with_blank
        order by unique_id, time
    """.format(
        cols1=cols1,
        cols2=cols2,
        cols3=cols3,
        cols4=cols4,
        cols5=cols5,
        cols6=cols6,
        cols7=cols7,
        cols8=cols8,
        user_count=user_count,
        base_date=base_date,
        range_days=range_days
    )
    return exec_query(query)

In [8]:
# base_date = '2020-06-08'
# range_days = 1
# user_count = 1000000
# gamiew_games = {
#     'dqw':'11',
#     'stella_arcana':'51',
#     'epick_seven':'58',
#     'kings_raid':'99',
#     'mitrasphere':'551'
# }
# train_df = pd.read_csv('200628_1000000_train_df.csv')
train_df = pd.read_csv('bq-results-20200629-142639-cjrddie7y8w5.csv')
# train_df = generate_train_df(base_date, range_days, genres, exec_query)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22877 entries, 0 to 22876
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   unique_id       22877 non-null  object 
 1   archive_id      22087 non-null  float64
 2   time            22877 non-null  object 
 3   hour            22877 non-null  int64  
 4   visit_days      20965 non-null  float64
 5   action          20965 non-null  float64
 6   tps_fps         20965 non-null  float64
 7   novel           20965 non-null  float64
 8   card            20965 non-null  float64
 9   casual          20965 non-null  float64
 10  simulation      20965 non-null  float64
 11  strategy        20965 non-null  float64
 12  sports          20965 non-null  float64
 13  puzzle          20965 non-null  float64
 14  board_table     20965 non-null  float64
 15  music           20965 non-null  float64
 16  racing          20965 non-null  float64
 17  action_rpg      20965 non-null 

In [9]:
# 特徴量の欠損値を1つ前の値で埋める
print(f"欠損値のある行数（穴埋め前）：{len(train_df)-sum(train_df['visit_days'].value_counts().values)}")
train_df[list(genres.keys())+['visit_days']] = train_df[list(genres.keys())+['visit_days']].fillna(method='ffill')
train_df[list(genres.keys())+['visit_days']] = train_df[list(genres.keys())+['visit_days']].fillna(method='bfill')
print(f"欠損値のある行数（穴埋め後）：{len(train_df)-sum(train_df['visit_days'].value_counts().values)}")

欠損値のある行数（穴埋め前）：1912
欠損値のある行数（穴埋め後）：0


In [10]:
train_df['cost'].value_counts()

 0    20965
-1     1912
Name: cost, dtype: int64

In [11]:
!pip install vowpalwabbit

In [12]:
from vowpalwabbit import pyvw
import random
import matplotlib.pyplot as plt

In [13]:
# This function modifies (context, action, cost, probability) to VW friendly format
def to_vw_example_format(context, actions, cb_label = None):
    if cb_label is not None:
        chosen_action, cost, prob = cb_label
    example_string = ""
    example_string += f"shared |User visit_days={context['visit_days']} hour={context['hour']} "
    example_string += ' '.join([action+'='+str(context[action]) for action in actions])
    example_string += '\n'
    for action in actions:
        if cb_label is not None and action == chosen_action:
            example_string += "0:{}:{} ".format(cost, prob)
        example_string += "|Action genre={} \n".format(action)
    #Strip the last newline
    return example_string[:-1]

def sample_custom_pmf(pmf):
    total = sum(pmf)
    scale = 1/total
    pmf = [x * scale for x in pmf]
    draw = random.random()
    sum_prob = 0.0
    for index, prob in enumerate(pmf):
        sum_prob += prob
        if(sum_prob > draw):
            return index, prob

def get_action(vw, context, actions, action = None):
    vw_text_example = to_vw_example_format(context,actions)
    pmf = vw.predict(vw_text_example)
    if action is not None:
        prob = pmf[actions.index(action)]
    else:
        chosen_action_index, prob = sample_custom_pmf(pmf)
        action = actions[chosen_action_index]
    return action, prob

In [16]:
def run_simulation(vw, df, actions, do_learn = True):
    cost_sum = 0.
    ctr = []

    for index, row in df.iterrows():
        context_user = {
            'visit_days' : row['visit_days'],
            'hour' : row['hour'],
        }
        context_genre = dict([[action,row[action]] for action in actions])
        context = dict(context_user, **context_genre)
        action = row['genre']
        action, prob = get_action(vw, context, actions, action)
        cost = row['cost']
        cost_sum += cost

        if do_learn:
            vw_format = vw.parse(to_vw_example_format(context, actions, (action, cost, prob)),pyvw.vw.lContextualBandit)
            vw.learn(vw_format)

        ctr.append(-1*cost_sum/(index+1))
    return ctr

In [17]:
def plot_ctr(df, ctr):
    plt.plot(range(1,len(df)+1), ctr)
    plt.xlabel('num_iterations', fontsize=14)
    plt.ylabel('ctr', fontsize=14)
    plt.ylim([0,1])

In [ ]:
actions = list(genres.keys())
df = train_df
vw = pyvw.vw("--cb_explore_adf -q UA --quiet --epsilon 0.2")
ctr = run_simulation(vw, df, actions)
plot_ctr(df, ctr)